In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Data path
import pathlib
DATA_FOLDER = pathlib.Path("/home/jovyan/work/Dan/data")

# Data visualisation
import pandas as pd

# Vectorize algotrithm
from gensim.models import Word2Vec


In [2]:
# Open pickle file and read data
PATH = DATA_FOLDER / 'pwdb/pickle/df_columns_labels.pkl'
df = pd.read_pickle(PATH)
# Set which data we want to use to train word2vec model
columns = df['Concatenated Data (clean)']
columns

0      [hardship, case, fund, safety, net, selfemploy...
1      [state, support, tourism, access, financeas, t...
2      [bank, guarantees, smes, oneperson, enterprise...
3      [emergency, measures, relating, shorttime, wor...
4      [airbus, agreement, making, unworked, hours, p...
                             ...                        
930    [taxation, measures, enterprisesas, consequenc...
931    [winwinloanthe, winwin, loan, system, encourag...
932    [support, measures, businesses, closed, since,...
933    [new, loans, available, enterprises, brusselst...
934    [financial, support, teleworkers, increase, te...
Name: Concatenated Data (clean), Length: 935, dtype: object

In [3]:
# Insert our data and set minimal word count to 10, and size of each word to 300 vectors
dfVec = Word2Vec(columns, window=5, min_count=10, size=300)
w2v_dict = {w: vec for w, vec in zip(dfVec.wv.index2word, dfVec.wv.syn0)}
w2v = dfVec.wv.syn0

In [4]:
# Test the result
dfVec.most_similar('president')

[('established', 0.9997254014015198),
 ('cabinet', 0.9996880888938904),
 ('parliament', 0.9995955228805542),
 ('amended', 0.9995614886283875),
 ('declaration', 0.9995083808898926),
 ('published', 0.9995009899139404),
 ('introduces', 0.999493420124054),
 ('czech', 0.9994755387306213),
 ('series', 0.999431312084198),
 ('croatia', 0.9994305968284607)]

In [18]:
# Save our trained model
# dfVec.save('/home/jovyan/work/Dan/data/pwdb/word2vec/df.model')